In [ ]:
from report.io.trajectory_parser import parse_trajectory
from report.data.geometry import Geometry

import pygeos
import matplotlib.pyplot as plt
from descartes import PolygonPatch
import numpy as np

# Setup geometry & measurement area

In [ ]:
ma = pygeos.polygons([(-1.5, 0), (-1.5, 5), (1.5, 5), (1.5, 0), (-1.5, 0)])
geometry = pygeos.polygons([(-9, 0), (-9, 5), (9, 5), (9, 0), (-9, 0)])

geometry_pita = Geometry(geometry)

traj = parse_trajectory("demos/uni-directional/traj_UNI_CORR_500_01.txt")

# Density

## Classic density

In [ ]:
from report.methods.density_calculator import compute_classic_density

classic_density = compute_classic_density(traj.data, ma)
classic_density

In [ ]:
classic_density.reset_index().plot.line(x='frame', y='classic density')

## Voronoi density

In [ ]:
from report.methods.density_calculator import compute_voronoi_density

density_voronoi, individual = compute_voronoi_density(traj.data, ma, geometry_pita)

In [ ]:
density_voronoi

In [ ]:
individual

In [ ]:
density_voronoi.reset_index().plot.line(x='frame', y='voronoi density')

## Comparision

In [ ]:
fig = plt.figure(figsize=(8, 6))
plt.plot(classic_density.reset_index().frame, classic_density["classic density"].values, label='classic')
plt.plot(density_voronoi.reset_index().frame, density_voronoi["voronoi density"], label='voronoi')
plt.xlabel('frame')
plt.ylabel('rho / 1/m^2')
plt.legend()
plt.grid()
plt.show()

## Plot voronoi cells

In [ ]:
import pandas as pd

frame_start = 1200

for frame in range(frame_start, frame_start+100, 20):
    df_frame = individual[individual.frame == frame]
    df_frame = pd.merge(traj.data, df_frame, on=["ID", "frame"])

    fig = plt.figure(figsize=(15, 20))
    ax1 = fig.add_subplot(111,aspect='equal')

    ax1.plot(*pygeos.to_shapely(geometry).exterior.xy, color='k')
    # ax1.plot(*pygeos.to_shapely(ma).exterior.xy, color='k')

    for _, row in df_frame.iterrows():
        p = ax1.plot(*pygeos.to_shapely(row[6]).exterior.xy)
        ax1.scatter(row[2], row[3])
        ax1.add_patch(PolygonPatch(pygeos.to_shapely(row[6]), alpha=0.1, zorder=2 , fc=p[-1].get_color(), ec='none'))
    ax1.set_xlabel('x')
    ax1.set_ylabel('y')
    plt.show()

# Velocity

In [ ]:
from report.methods.velocity_calculator import compute_individual_velocity

individual_speed = compute_individual_velocity(traj.data, traj.frame_rate, 5)
individual_speed

In [ ]:
individual_speed.reset_index().plot.scatter(x='frame', y='speed')

In [ ]:
direction = np.array([-1, 0])
individual_speed_direction = compute_individual_velocity(traj.data, traj.frame_rate, 5, direction)
individual_speed_direction

In [ ]:
individual_speed.reset_index().plot.scatter(x='frame', y='speed')

## Mean speed (all)

In [ ]:
from report.methods.velocity_calculator import compute_mean_velocity_per_frame

mean_speed, individual_speed = compute_mean_velocity_per_frame(traj.data, traj.frame_rate, 5)
mean_speed

In [ ]:
mean_speed.reset_index().plot.scatter(x='frame', y='speed')

## Mean speed (in measurement area)

In [ ]:
from report.methods.method_utils import get_peds_in_area
from report.methods.velocity_calculator import compute_mean_velocity_per_frame

peds_in_area = get_peds_in_area(traj.data, ma)
mean_speed_area, individual_speed_area = compute_mean_velocity_per_frame(peds_in_area, traj.frame_rate, 5, np.array([-1, 0]))

In [ ]:
mean_speed_area.reset_index().plot.scatter(x='frame', y='speed')

## Mean speed (in measurement area)

In [ ]:
from report.methods.method_utils import get_peds_in_area
from report.methods.velocity_calculator import compute_mean_velocity_per_frame

peds_in_area = get_peds_in_area(traj.data, ma)
mean_speed_area, individual_speed_area = compute_mean_velocity_per_frame(peds_in_area, traj.frame_rate, 5)

In [ ]:
mean_speed_area.reset_index().plot.scatter(x='frame', y='speed')